In [1]:
from transformers import DetrFeatureExtractor, DetrForObjectDetection
import torch
from PIL import Image
import requests
import shutil
import os
import matplotlib.pyplot as plt
from bing_image_downloader import downloader
import shutil

In [3]:
# Get images of cars in traffic
if os.path.exists("images/cars in highway"):
   shutil.rmtree("images/cars in highway")
os.mkdir("images/cars in highway")
downloader.download("cars in highway", limit=20,  output_dir='images',
                    adult_filter_off=True, force_replace=False, timeout=60, verbose=False, filter="photo")


[%] Downloading Images to /mnt/d/DSLab/ML/EigenCars/images/cars in highway


KeyboardInterrupt: 

In [43]:
# function to detect cars in a given image
def detect_cars(image_path, confidence=0.9, threshold=32):
	image=Image.open(image_path)
	feature_extractor = DetrFeatureExtractor.from_pretrained(
			"facebook/detr-resnet-50")
	model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

	inputs = feature_extractor(images=image, return_tensors="pt")
	outputs = model(**inputs)

	# convert outputs (bounding boxes and class logits) to COCO API
	target_sizes = torch.tensor([image.size[::-1]])
	results = feature_extractor.post_process(outputs, target_sizes=target_sizes)[0]

	# let's only keep detections with score > confidence
	rects=[]
	for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
			box = [round(i, 2) for i in box.tolist()]
			if score > confidence:
				# check if rect is larger than threshold number of pixels in both dimensions
				if (box[2]-box[0])>threshold and (box[3]-box[1])>threshold:
					rects.append(box)
	return rects

In [44]:
# function to create new cropped images of cars
def crop_cars(image_path,index):
  rects=detect_cars(image_path)
  image=Image.open(image_path)
  for i, rect in enumerate(rects):
    x1, y1, x2, y2 = rect
    car = image.crop((x1, y1, x2, y2))
    # create a new image with the car
    if not os.path.exists("./cars"):
      os.mkdir("cars")
    car.save("./cars/"+str(index)+"_"+str(i)+".png")

In [45]:
# loop through all images in the images folder
for image_path in os.listdir("images/cars in highway"):
  crop_cars("images/cars in highway/"+image_path, image_path)

/home/abishek/.local/lib/python3.10/site-packages/transformers/models/detr/image_processing_detr.py:773: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(
/home/abishek/.local/lib/python3.10/site-packages/transformers/models/detr/image_processing_detr.py:1236: FutureWarning: `post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`
  warnings.warn(


In [46]:
# Get images of people in traffic
if os.path.exists("images/people in streets"):
   shutil.rmtree("images/people in streets")
os.mkdir("images/people in streets")
downloader.download("people in streets", limit=20,  output_dir='images',
                    adult_filter_off=True, force_replace=False, timeout=60, verbose=False, filter="photo")


[%] Downloading Images to /mnt/d/DSLab/ML/EigenCars/images/people in streets
[!] Issue getting: https://dm0qx8t0i9gc9.cloudfront.net/thumbnails/video/D8qa-2E/large-group-of-people-crossing-crowded-city-street-new-york-scenery_hhobtpsw_thumbnail-1080_01.png
[!] Error:: <urlopen error _ssl.c:980: The handshake operation timed out>
[!] Issue getting: https://dm0qx8t0i9gc9.cloudfront.net/thumbnails/video/D8qa-2E/diverse-people-crossing-street-in-new-york-city_shmhurjd_thumbnail-1080_01.png
[!] Error:: <urlopen error _ssl.c:980: The handshake operation timed out>
[!] Issue getting: https://dm0qx8t0i9gc9.cloudfront.net/thumbnails/video/D8qa-2E/new-york-city-streets-scene-crowd-of-business-people-commuting-background_rbqhd0jv_thumbnail-1080_02.png
[!] Error:: <urlopen error _ssl.c:980: The handshake operation timed out>


[%] Done. Downloaded 20 images.
